# Wedding FOM: To Optimize Cost Or Not?
T-Lind

In [3]:
import json
from timeit import default_timer as timer
import matplotlib.pyplot as plt

from ptcr2.fom import FOM

config_file = 'samples/wedding_fom.json'

with open(config_file) as f:
    spec = json.loads(f.read())


In [4]:
wedding_fom_no_cost = FOM()
start = timer()
wedding_fom_no_cost.compute_optimal_policy(spec)
end = timer()

print('Time elapsed to compute no-cost optimal policy: ', end - start)

Time elapsed to compute no-cost optimal policy:  91.3416663


In [5]:
wedding_fom_cost = FOM()
start = timer()
wedding_fom_cost.compute_optimal_policy(spec, cost_based=True)
end = timer()

print('Time elapsed to compute cost-based optimal policy: ', end - start)

COMPUTING COST BASED POLICY!!!
Transition from 0 to 1 with cost 1
Transition from 0 to 2 with cost 2
Transition from 0 to 3 with cost 3
Transition from 0 to 5 with cost 5


IndexError: list index out of range

Plots the training runs of the general algorithms. This can be helpful to determine how various factors might affect the training process.

In [ ]:
plt.title('Difference Measured While Computing Optimal Policy')
plt.xlabel('Iteration')
plt.ylabel('Difference')
plt.plot(wedding_fom_no_cost.computed_policy['diff_tracker'], 'g', label='Step-based')
plt.plot(wedding_fom_cost.computed_policy['diff_tracker'], 'b', label='Cost-based')
plt.axhline(wedding_fom_no_cost.epsilon, linestyle='--', color='r', label='Epsilon of Convergence')  # Same epsilon for both since they come from same spec
plt.legend()

Prints the result of one simulation run for both the no-cost and cost algorithm. This can be used to compare how much better the cost-based general algorithm is.

In [9]:
# Example run
result_no_cost = wedding_fom_no_cost.simulate_general_and_greedy_algorithms()  # Spec not needed since we're computing optimal policy separately
print(result_no_cost)
result_cost = wedding_fom_cost.simulate_general_and_greedy_algorithms()  # Spec not needed since we're computing optimal policy separately
print(result_cost)

{'general_algorithm': {'steps': 15, 'story': 's3d2s3c3d12', 'total_cost': 7}, 'greedy_algorithm': {'steps': 15, 'story': 's3d2s3c3d12', 'total_cost': 7}}


In [11]:
from tqdm import tqdm

# Now, simulate the algorithms across 1000 runs
n_runs = 10_000

no_cost_steps = []
no_cost_costs = []

cost_steps = []
cost_costs = []

for _ in tqdm(range(n_runs)):
    result_no_cost = wedding_fom_no_cost.simulate()
    no_cost_steps.append(result_no_cost['steps'])
    no_cost_costs.append(result_no_cost['total_cost'])

    result_cost = wedding_fom_cost.simulate()
    cost_steps.append(result_cost['steps'])
    cost_costs.append(result_cost['total_cost'])


100%|██████████| 10000/10000 [00:39<00:00, 250.89it/s]


In [ ]:
# Plot the two histograms, with cost/no-cost distributions in different colors
plt.hist(no_cost_steps, bins=50, alpha=0.5, label='No-Cost')
plt.hist(cost_steps, bins=50, alpha=0.5, label='Cost-Optimized')

plt.legend()
plt.title('Distribution of Steps Taken by Costless and Cost-Optimized Algorithms')
plt.xlabel('Steps')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Plot the two histograms, with cost/no-cost distributions in different colors
plt.hist(no_cost_costs, bins=20, alpha=0.5, label='No-Cost')
plt.hist(cost_costs, bins=20, alpha=0.5, label='Cost-Optimized')

plt.legend()
plt.title('Distribution of Costs Incurred by Costless and Cost-Optimized Algorithms')
plt.xlabel('Cost')
plt.ylabel('Frequency')
plt.show()

In [14]:
import numpy as np

# Print 5-number summary + mean for general and greedy steps taken
print('Costless Algorithm Steps Summary:')
print('Min:', np.min(no_cost_steps))
print('Q1:', np.percentile(no_cost_steps, 25))
print('Median:', np.median(no_cost_steps))
print('Mean:', np.mean(no_cost_steps))
print('Q3:', np.percentile(no_cost_steps, 75))
print('Max:', np.max(no_cost_steps))

print('\nCost-Optimized Algorithm Steps Summary:')
print('Min:', np.min(cost_steps))
print('Q1:', np.percentile(cost_steps, 25))
print('Median:', np.median(cost_steps))
print('Mean:', np.mean(cost_steps))
print('Q3:', np.percentile(cost_steps, 75))
print('Max:', np.max(cost_steps))

# Print 5-number summary + mean for general and greedy costs incurred
print('\nCostless Algorithm Costs Summary:')
print('Min:', np.min(no_cost_costs))
print('Q1:', np.percentile(no_cost_costs, 25))
print('Median:', np.median(no_cost_costs))
print('Mean:', np.mean(no_cost_costs))
print('Q3:', np.percentile(no_cost_costs, 75))
print('Max:', np.max(no_cost_costs))

print('\nCost-Optimized Algorithm Costs Summary:')
print('Min:', np.min(cost_costs))
print('Q1:', np.percentile(cost_costs, 25))
print('Median:', np.median(cost_costs))
print('Mean:', np.mean(cost_costs))
print('Q3:', np.percentile(cost_costs, 75))
print('Max:', np.max(cost_costs))

General Algorithm Steps Summary:
Min: 5
Q1: 18.0
Median: 26.0
Mean: 30.4623
Q3: 37.0
Max: 173

Greedy Algorithm Steps Summary:
Min: 5
Q1: 20.0
Median: 28.0
Mean: 32.4772
Q3: 40.0
Max: 173

General Algorithm Costs Summary:
Min: 1
Q1: 8.0
Median: 10.0
Mean: 10.0512
Q3: 12.0
Max: 18

Greedy Algorithm Costs Summary:
Min: 2
Q1: 8.0
Median: 10.0
Mean: 10.1975
Q3: 12.0
Max: 19


In [31]:
from scipy.stats import ttest_ind

alpha = 0.05

# We want to check if both the number of steps and cost is lower for the cost-optimized algorithm than the costless algorithm
t_stat, p_val = ttest_ind(cost_steps, no_cost_steps, equal_var=False, alternative='less')
print("Performing 2-sample t-test to compare costless and cost-optimized algorithms:")
print('T-Statistic:', t_stat)
print('P-Value:', p_val)


if p_val < alpha:
    print('Reject the null hypothesis: The cost-optimized algorithm is significantly lower in steps than the costless algorithm.')
else:
    print('Fail to reject the null hypothesis: The cost-optimized algorithm is not significantly lower in steps than the costless algorithm.')
    
# Perform an independent 2-sample t-test to determine if the cost-optimized algorithm is significantly lower in cost than the costless algorithm
t_stat, p_val = ttest_ind(cost_costs, no_cost_costs, equal_var=False, alternative='less')
print("\nPerforming 2-sample t-test to compare costless and cost-optimized algorithms:")
print('T-Statistic:', t_stat)
print('P-Value:', p_val)

Performing 2-sample t-test to compare general and greedy algorithms:
T-Statistic: -26.963914303489023
P-Value: 3.80146655189153e-160
Reject the null hypothesis: The general algorithm is significantly better than the greedy algorithm.

Performing 2-sample t-test to compare general and greedy algorithms:
T-Statistic: -12.844215714201036
P-Value: 4.796951818527035e-38
